In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

%matplotlib inline 

# All iNaturalist Records of Lobelia Spicata inside of the U.S.
Spicata is the target species. It has a similar number of U.S. observations as does Yellow Lady's Slipper and Spring Ladies' Tresses, yet those species have higher level conservation status than Spicata so, if studied, Spicata's latitude and longitude won't be obscured via a taxon geoprivacy, making it data that I can use for validation of models.

To simulate how I imagine someone might target an endangered species, I pulled all observations of Spicata and from there a list of users whose records will have "clues" about the location of the Spicata plants. For validation purposes, I also need to make sure the user's observations had an actual public positional accuracy of at least 30 m. 

In [2]:
df_spicata = pd.read_csv("../../data/observations_spicata.csv")
df_spicata.head()

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,user_name,created_at,updated_at,...,taxon_supertribe_name,taxon_tribe_name,taxon_subtribe_name,taxon_genus_name,taxon_genushybrid_name,taxon_species_name,taxon_hybrid_name,taxon_subspecies_name,taxon_variety_name,taxon_form_name
0,82406,"May 20, 2012 16:44",2012-05-20,2012-05-20 21:44:00 UTC,Central Time (US & Canada),604,eric_hunt,Eric Hunt,2012-05-23 22:43:55 UTC,2020-12-09 10:50:50 UTC,...,NaN,NaN,NaN,Lobelia,NaN,Lobelia spicata,NaN,NaN,NaN,NaN
1,82408,"May 20, 2012 16:19",2012-05-20,2012-05-20 21:19:00 UTC,Central Time (US & Canada),604,eric_hunt,Eric Hunt,2012-05-23 22:46:36 UTC,2020-12-09 10:50:51 UTC,...,NaN,NaN,NaN,Lobelia,NaN,Lobelia spicata,NaN,NaN,NaN,NaN
2,87039,Sun Jun 03 2012 09:52:31 GMT-0400 (EDT),2012-06-03,2012-06-03 13:52:31 UTC,Eastern Time (US & Canada),477,loarie,Scott Loarie,2012-06-04 05:25:45 UTC,2019-07-02 19:37:38 UTC,...,NaN,NaN,NaN,Lobelia,NaN,Lobelia spicata,NaN,NaN,NaN,NaN
3,92772,"June 16, 2012 05:40",2012-06-16,2012-06-16 12:40:00 UTC,Pacific Time (US & Canada),477,loarie,Scott Loarie,2012-06-18 18:04:31 UTC,2015-10-08 14:36:10 UTC,...,NaN,NaN,NaN,Lobelia,NaN,Lobelia spicata,NaN,NaN,NaN,NaN
4,195645,2008-07-06,2008-07-06,NaN,Eastern Time (US & Canada),12610,susanelliott,Susan Elliott,2013-02-10 18:04:29 UTC,2020-02-19 21:15:23 UTC,...,NaN,NaN,NaN,Lobelia,NaN,Lobelia spicata,NaN,NaN,NaN,NaN


In [3]:
df_spicata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4547 entries, 0 to 4546
Data columns (total 67 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                4547 non-null   int64  
 1   observed_on_string                4538 non-null   object 
 2   observed_on                       4538 non-null   object 
 3   time_observed_at                  4437 non-null   object 
 4   time_zone                         4547 non-null   object 
 5   user_id                           4547 non-null   int64  
 6   user_login                        4547 non-null   object 
 7   user_name                         3079 non-null   object 
 8   created_at                        4547 non-null   object 
 9   updated_at                        4547 non-null   object 
 10  quality_grade                     4547 non-null   object 
 11  license                           3489 non-null   object 
 12  url   

# Comparing public positional accuracy to positional accuracy

Positional accuracy is a kind of confidence measure in the lat/lon provided. An observation is said to be within x meters of the given lat/lon where x is the positional accuracy. A positional accuracy of 30 m or less is considered "research standard" [according to iNaturalist](https://www.inaturalist.org/posts/2035-observation-location-accuracy). However, the downloaded data has two categories for positional accuracy--one labelled "positional accuracy" and one labelled "public positional accuracy". I was not able to find an explanation about the difference on the website so I've compared below with the data. Because I want to use spicata observations where the lat/lon is tied to a positional accuracy of 30 m or less, it's important to know which of the two standards to use, since they do show they differ. 

In [4]:
# number of spicata observations with public pos acc of 30 m or less
(df_spicata["public_positional_accuracy"] <= 30).sum()

2076

In [5]:
# number of spicata observations with pos acc of 30 m or less
(df_spicata["positional_accuracy"] <= 30).sum()

2304

In [6]:
# suspect the 228 difference between two counts is due to geoprivacy
# checking that suspicion 

#boolean table for all rows--True if positional accuracy is at least 30, but public pos acc is greater
truth_table = (df_spicata[df_spicata["positional_accuracy"] <= 30]["public_positional_accuracy"] > 30) 
# an array for those index numbers where the condition is True
indices_for_diff = truth_table[truth_table == True].index
# len function confirms I've isolated the 228 rows
len(indices_for_diff)

228

In [7]:
# a count of geoprivacy for those entries shows all of them have "obscured" geoprivacy
df_spicata.loc[indices_for_diff, "id":"coordinates_obscured"]["geoprivacy"].value_counts()

obscured    228
Name: geoprivacy, dtype: int64

All differences between public pos acc and pos acc are accounted for by an obscured geoprivacy. I am going to extrapolate that information to state that I believe that "positional accuracy" is the accuracy attached to the "private latitude" and "private longitude"--values which are not available to me. Whereas "public positional accuracy" is the accuracy attached to the "latitude" and "longitude" values--which are publicly available to me. So the "public positional accuracy" is the metric I need to focus on.

# Constructing a Dataset around Spicata

In [8]:
# list of all users in US who posted spicata with public positional accuracy <= 30
spicata_users = df_spicata[df_spicata["public_positional_accuracy"] <= 30]["user_id"].unique().tolist()
len(spicata_users)

1245

Ideally, I would get to work with all of the data for the full number of users in the set above. And that may be possible as I learn more ways to access the data. But given my current skillset and the way iNaturalist's data query page works, I am using a brute force method of choosing 10 random users at a time from the list of generated users and pulling each user's records by query from the iNaturalist query page until I have a set to explore that has at least 100,000 data points AND at least 100 Spicata records. 

In [9]:
# 10 users chosen at random using random.shuffle ALL HAVE BEEN INCLUDED
to_remove = [ 635041, 2588524,  923056, 18434,  318468, 1549697, 3583533,
       1679129, 2047965, 2570804]

In [10]:
for id in to_remove:
    spicata_users.remove(id)
len(spicata_users)

1235

In [11]:
# next 10 random users: HAS BEEN INCLUDED
to_remove2 = [787855, 1892152, 542981, 656158, 2248142, 780600, 4659453, 2336149, 3512034, 1773265]
to_remove.extend(to_remove2)

In [12]:
for id in to_remove2:
    spicata_users.remove(id)
len(spicata_users)

1225

In [13]:
# HAS BEEN INCLUDED
to_remove3 = [2889644, 2718421, 805798, 518143, 1237684, 4531483, 818667, 1081392, 2585152, 1401675]
to_remove.extend(to_remove3)

In [14]:
for id in to_remove3:
    spicata_users.remove(id)
len(spicata_users)

1215

In [15]:
# HAS BEEN INCLUDED
to_remove4 = [328268, 557161,2808877,2907199,1234210,1006938,1750165,2154312,1531120,1834316]
to_remove.extend(to_remove4)

In [16]:
for id in to_remove4:
    spicata_users.remove(id)
len(spicata_users)

1205

In [17]:
# HAS BEEN INCLUDED
to_remove5 = [1511270, 1237597, 5800322, 416976, 5669487, 3823990, 4736884, 5241676, 1149437, 1916723]
to_remove.extend(to_remove5)

In [18]:
for id in to_remove5:
    spicata_users.remove(id)
len(spicata_users)

1195

In [19]:
# HAS BEEN INCLUDED
to_remove6 = [436529, 1262660, 674664, 477, 1976181, 475357, 1883226, 3062686, 2125177, 265604]
to_remove.extend(to_remove6)


In [20]:
for id in to_remove6:
    spicata_users.remove(id)
len(spicata_users)

1185

In [21]:
to_remove7 = [1804605, 11792, 434504, 604, 3199368, 1872981, 1387258, 392818, 1635190, 289544]
to_remove.extend(to_remove7)

In [22]:
for id in to_remove7:
    spicata_users.remove(id)
len(spicata_users)

1175

In [23]:
#spicata_0071
to_remove8 = [1576744, 2172610, 3701063, 953893, 1054497, 5544835, 1930769, 214446, 553407, 2704630]
to_remove.extend(to_remove8)

In [24]:
for id in to_remove8:
    spicata_users.remove(id)
len(spicata_users)

1165

In [25]:
#spicata_0072 and 0073
to_remove9 = [1467084,
 4132372,
 147091,
 18446,
 246712,
 1781904,
 150454,
 582522,
 1686639,
 4129746]
to_remove.extend(to_remove9)

In [26]:
for id in to_remove9:
    spicata_users.remove(id)
len(spicata_users)

1155

In [27]:
#spicata_0074
to_remove10 = [1242401,
 80782,
 1308403,
 471690,
 5814233,
 2963491,
 1796136,
 894957,
 4672797,
 1518270]
to_remove.extend(to_remove10)
len(to_remove)

100

In [28]:
for id in to_remove10:
    spicata_users.remove(id)
len(spicata_users)

1145

In [29]:
#spicata_0075
to_remove11 = [2375374,
 2860446,
 463066,
 851197,
 845209,
 521220,
 307610,
 938102,
 3534207,
 2399521]
to_remove.extend(to_remove11)
len(to_remove)

110

In [30]:
for id in to_remove11:
    spicata_users.remove(id)
len(spicata_users)

1135

In [31]:
# spicata_0076
to_remove12 = [691967,
 25564,
 1590417,
 3074897,
 552204,
 777160,
 2468086,
 6094378,
 1464758,
 2707492]
to_remove.extend(to_remove12)
len(to_remove)

120

In [32]:
for id in to_remove12:
    spicata_users.remove(id)
len(spicata_users)

1125

In [33]:
# spicata_0077
to_remove13 = [2914217,
 1141686,
 1910808,
 395895,
 44631,
 580546,
 1670419,
 643129,
 827528,
 1820997,
 747703,
 149431,
 4476908,
 769013,
 5498368,
 2929497,
 3183213,
 183583,
 5410150,
 2772034]
to_remove.extend(to_remove13)
len(to_remove)

140

In [34]:
for id in to_remove13:
    spicata_users.remove(id)
len(spicata_users)

1105

In [35]:
# spicata_0078
to_remove14 = [4730192,
 5436559,
 765149,
 950162,
 538744,
 35203,
 128628,
 18604,
 2182686,
 246309]
to_remove.extend(to_remove14)
len(to_remove)

150

In [36]:
for id in to_remove14:
    spicata_users.remove(id)
len(spicata_users)

1095

In [37]:
#spicata_0079
to_remove15 = [8882,
 454217,
 920858,
 1020281,
 1935198,
 318214,
 4633627,
 138422,
 3162904,
 346073]
to_remove.extend(to_remove15)
len(to_remove)

160

In [38]:
#spicata_0080
for id in to_remove15:
    spicata_users.remove(id)
len(spicata_users)

1085

In [39]:
to_remove16 = [2762217,
 4697264,
 1110621,
 4571361,
 1459875,
 2847279,
 6917800,
 4591168,
 2082693,
 2682906,
 4396950,
 1174034,
 28982,
 1289977,
 9394,
 375183,
 1643556,
 2912158,
 68132,
 1247211]
to_remove.extend(to_remove16)
len(to_remove)

180

In [40]:
for id in to_remove16:
    spicata_users.remove(id)
len(spicata_users)

1065

In [41]:
#spicata_0081
to_remove17 = [920561,
 4009332,
 2357773,
 1900488,
 1837096,
 2419279,
 4382878,
 59391,
 934551,
 2803134,
 2866704,
 27450,
 708334,
 977888,
 507882,
 3140204,
 142886,
 320666,
 1189913,
 3185240]
to_remove.extend(to_remove17)
len(to_remove)

200

In [42]:

for id in to_remove17:
    spicata_users.remove(id)
len(spicata_users)

1045

In [43]:
#spicata_0082
to_remove18 = [257635,
 1002988,
 898587,
 2322709,
 1038812,
 5281113,
 138597,
 2013202,
 3932812,
 511036,
 783913,
 2648010,
 1249776,
 1518446,
 4311442,
 111150,
 442729,
 604764,
 2038605,
 3726501]
to_remove.extend(to_remove18)
len(to_remove)

220

In [44]:
for id in to_remove18:
    spicata_users.remove(id)
len(spicata_users)

1025

In [45]:
# spicata_0083
to_remove19 = [2686821,
 3490685,
 317,
 75312,
 1749181,
 4444244,
 2240459,
 2170071,
 4162472,
 220924]
to_remove.extend(to_remove19)
len(to_remove)


230

In [46]:
for id in to_remove19:
    spicata_users.remove(id)
len(spicata_users)

1015

In [47]:
# spicata_0084
to_remove20 = [5822396,
 574034,
 5958975,
 704781,
 77652,
 1026528,
 1416798,
 2850982,
 2757482,
 1258646,
 2275107,
 1820905,
 6922111,
 5058418,
 1269999,
 1713200,
 1488975,
 1158036,
 4538108,
 5521453]
to_remove.extend(to_remove20)
len(to_remove)

250

In [48]:
for id in to_remove20:
    spicata_users.remove(id)
len(spicata_users)

995

In [49]:
#spicata_0085
to_remove21 = [208028,
 898475,
 2748753,
 200990,
 12194,
 2886615,
 690232,
 607706,
 3712023,
 920250,
 6169168,
 2075470,
 492160,
 428492,
 453256,
 5698210,
 1790408,
 1722298,
 119175,
 2590765]
to_remove.extend(to_remove21)
len(to_remove)

270

In [50]:
for id in to_remove21:
    spicata_users.remove(id)
len(spicata_users)

975

In [51]:
# spicata_0086
to_remove22 = [4812839,
 4211051,
 3130064,
 2982973,
 238235,
 3274391,
 2450320,
 4787148,
 5455991,
 4805499,
 3449782,
 626071,
 5445479,
 1631432,
 4669975,
 2635503,
 3293143,
 303051,
 3229159,
 2140372,
 4979062,
 2725921,
 3196688,
 1863615,
 4507493,
 879980,
 1794130,
 5331126,
 2619368,
 665398]
to_remove.extend(to_remove22)
len(to_remove)

300

In [52]:
for id in to_remove22:
    spicata_users.remove(id)
len(spicata_users)

945

In [53]:
#reads in the csv files from separate users and puts them together in one data frame
data_sp = pd.read_csv("../../data/spicata_0001.csv")
for i in range(2,87):
    if i < 10:
        file = "../../data/spicata_000" + str(i) +".csv"
    elif i < 100:
        file = "../../data/spicata_00" + str(i) +".csv"
    data_sp = pd.concat([data_sp, pd.read_csv(file)])

data_sp = data_sp.reset_index(drop = True)

C:\Users\lisa_\AppData\Local\Temp\ipykernel_47376\3626985300.py:8: DtypeWarning: Columns (15,63) have mixed types. Specify dtype option on import or set low_memory=False.
  data_sp = pd.concat([data_sp, pd.read_csv(file)])
C:\Users\lisa_\AppData\Local\Temp\ipykernel_47376\3626985300.py:8: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  data_sp = pd.concat([data_sp, pd.read_csv(file)])
C:\Users\lisa_\AppData\Local\Temp\ipykernel_47376\3626985300.py:8: DtypeWarning: Columns (14,29,32,33,34,63,66) have mixed types. Specify dtype option on import or set low_memory=False.
  data_sp = pd.concat([data_sp, pd.read_csv(file)])
C:\Users\lisa_\AppData\Local\Temp\ipykernel_47376\3626985300.py:8: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  data_sp = pd.concat([data_sp, pd.read_csv(file)])
C:\Users\lisa_\AppData\Local\Temp\ipykernel_47376\3626985300.py:8: DtypeWarning: Columns (14,15,34,

In [54]:
# double-checking that the set has the expects number of users
data_sp["user_id"].nunique()

300

In [55]:
data_sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071656 entries, 0 to 1071655
Data columns (total 67 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   id                                1071656 non-null  int64  
 1   observed_on_string                1070256 non-null  object 
 2   observed_on                       1070218 non-null  object 
 3   time_observed_at                  1052980 non-null  object 
 4   time_zone                         1071656 non-null  object 
 5   user_id                           1071656 non-null  int64  
 6   user_login                        1071656 non-null  object 
 7   user_name                         854568 non-null   object 
 8   created_at                        1071656 non-null  object 
 9   updated_at                        1071656 non-null  object 
 10  quality_grade                     1071656 non-null  object 
 11  license                           919

# Understanding num_identification and Related Columns

Used the below data to cross-reference iNaturalist observations on the sight itself to better be able to see the full story behind the numbers in these columns. What I deduced about what the four columns in question mean are described in the data dictionary in my README file.

In [56]:
data_sp["num_identification_agreements"].describe()

count    1.071656e+06
mean     1.072276e+00
std      9.210026e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      3.500000e+01
Name: num_identification_agreements, dtype: float64

In [57]:
data_sp[data_sp["num_identification_agreements"]==15][["user_id", "species_guess", "taxon_species_name", "num_identification_agreements", "num_identification_disagreements"]]

,user_id,species_guess,taxon_species_name,num_identification_agreements,num_identification_disagreements
63140,1237597,Mountain Lion,Puma concolor,15,0
425907,395895,Double-crested Cormorant,Nannopterum auritum,15,0


In [58]:
data_sp["num_identification_disagreements"].describe()

count    1.071656e+06
mean     4.550901e-03
std      6.958349e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.000000e+00
Name: num_identification_disagreements, dtype: float64

In [59]:
data_sp[data_sp["num_identification_disagreements"]>0][["user_id", "species_guess", "taxon_species_name", "num_identification_agreements", "num_identification_disagreements"]]

,user_id,species_guess,taxon_species_name,num_identification_agreements,num_identification_disagreements
206,635041,closed bottle gentian,Gentiana andrewsii,3,1
735,2588524,Mining Bees,NaN,3,1
741,2588524,NaN,Arge similis,0,1
833,2588524,NaN,NaN,0,1
955,2588524,Andrena,NaN,2,1
...,...,...,...,...,...
1066484,5455991,Silvery Checkerspot,Chlosyne nycteis,3,1
1067076,4979062,Timulla barbata,Timulla barbata,3,1
1067981,665398,Plant Bugs,NaN,3,1
1069516,5455991,Segmented Worms,NaN,3,1


In [60]:
# I supsected that "species_guess" was not always at the species level, used the most generic category possible to test
(data_sp["species_guess"] == "Life").sum()

224

In [61]:
data_sp[data_sp["species_guess"] == "Life"][["user_id", "species_guess", "taxon_species_name", "num_identification_agreements", "num_identification_disagreements"]]

,user_id,species_guess,taxon_species_name,num_identification_agreements,num_identification_disagreements
4283,2588524,Life,NaN,2,0
5409,2588524,Life,NaN,0,0
11114,2588524,Life,NaN,2,0
11587,2588524,Life,NaN,0,0
14008,1679129,Life,NaN,1,0
...,...,...,...,...,...
990071,920250,Life,NaN,2,0
1006775,2886615,Life,NaN,2,0
1028169,5698210,Life,NaN,1,0
1060895,626071,Life,NaN,2,0


# Cleaning the Dataset

### Dropping Columns and Rows

__Dropping columns__
I originally downloaded all data columns available via the iNaturalist query page. However, many are not of immediate interest, especially as my first model will involve basic interpolation. Other columns may be of interest later, but for now I'm choosing 21 of the 66 columns for EDA and then my first attempts at modeling. 

__Dropping missing lat/lon rather than attempting to impute because:__
1) accurate lat/lon is essential to this analysis
2) accurately imputing from other clues would be time intensive
3) null lat/lon makes up less than 1% of the rows, so dropping doesn't significantly hurt my dataset

__Dropping missing "time_observed_on" rows because:__
1) much of my analysis is based on this value
2) null values is less than 5% of the rows, so dropping doesn't hurt my dataset
*HOWEVER: it is worth noting that there are two choices for imputation that could be explored:*
1) impute the "time_created_at" value AND also working with the "time_created_at" value for ordering is another way to consider ordering the data.
2) order the data by "time_created_at" and then impute a "time_observed_at" that falls between the "time_observed_at" tows on either side of the null row (assuming the same user)
But it is my assessment that this will not impact my dataset enough to make it worth the effort.

In [62]:
#columns of interest
to_keep = ['id', "observed_on_string", "observed_on", 'time_observed_at', "time_zone", 'user_id', 'created_at',
       'quality_grade', "url", "image_url", "sound_url", 'num_identification_agreements',
       'num_identification_disagreements', 'captive_cultivated',
       'latitude', 'longitude',
       'positional_accuracy', 'public_positional_accuracy', 'geoprivacy',
       'taxon_geoprivacy', 'coordinates_obscured', 'species_guess', 'scientific_name', 'common_name',
       'taxon_kingdom_name','taxon_genus_name',
      'taxon_species_name']
data_sp = data_sp[to_keep]



In [63]:
# dropping missing "time_observed_at" rows and confirming
data_sp.dropna(subset=['time_observed_at'], inplace=True)
print(f'Number of null time_observed_at entries = {data_sp[data_sp["time_observed_at"].isnull()].shape[0]}')

Number of null time_observed_at entries = 0


In [64]:
# dropping missing latitude/longitude rows and confirming
# for most part, these nulls usually are the same
# but occasional strangeness requires choosing to drop category with most nulls
if data_sp[data_sp["latitude"].isnull()].shape[0] >= data_sp[data_sp["longitude"].isnull()].shape[0]:
    drop_name = "latitude"
else:
    drop_name = "longitude"
data_sp.dropna(subset=[drop_name], inplace=True)
print(f'Number of null time_observed_at entries = {data_sp[data_sp[drop_name].isnull()].shape[0]}')

Number of null time_observed_at entries = 0


### Boolean

__geoprivacy__
has only an "obscured" value--the rest are null. Simple boolean.

__taxon_geoprivacy__
has two values "open" and "obscured" but also many nulls, however cross-referencing with the "coordinates obscured" column shows that any null values in "taxon_geoprivacy" where coordinates are marked as obscured are entirely accounted for by geoprivacy, so there aren't any obscured values noted in the database that need to be accounted for within the nulls of "taxon_geoprivacy" so I will be treating this column as boolean with "taxon_geoprivacy_obscured" or not. 


In [65]:
data_sp["geoprivacy"].value_counts() #to boolean

obscured    60447
private         1
Name: geoprivacy, dtype: int64

In [66]:
# this code demonstrated during Dec 19th kickoff and had short runtime compared to others
# this way of using .apply and lambda isn't yet comfortable/natural to me

data_sp["geoprivacy_obscured"] = data_sp["geoprivacy"].apply(lambda x: 1 if x == "obscured" else 0)
#retitles boolean for clarity to distinguish between taxon_geoprivacy "open" and drops original column
data_sp.drop("geoprivacy", axis = 1, inplace = True)

In [67]:
data_sp["geoprivacy_obscured"].value_counts()

0    989704
1     60447
Name: geoprivacy_obscured, dtype: int64

In [68]:
data_sp["taxon_geoprivacy"].value_counts() # to dummy

open        227487
obscured     16793
Name: taxon_geoprivacy, dtype: int64

In [69]:
# shows how many of null taxon_geoprivacy are marked as "coordinates_obscured" or not
na_taxon = data_sp[data_sp["taxon_geoprivacy"].isna()]
(na_taxon["coordinates_obscured"] == True).sum()

43598

In [70]:
# shows that same number is accounted for by "geoprivacy_obscured"
na_taxon[na_taxon["coordinates_obscured"] == True]["geoprivacy_obscured"].sum()

43597

In [71]:
# boolean
data_sp["taxon_geoprivacy_obscured"] = data_sp["taxon_geoprivacy"].apply(lambda x: 1 if x == "obscured" else 0)
data_sp.drop("taxon_geoprivacy", axis =1, inplace = True)

### Imputation

__taxon_kingdom_name__
For now I am imputing the string "not stated" into the null values. However, if future model building would benefit from hot-encoding, the bulk of the data is in the Animal, Plant or Fungi kingdoms so it is unlikely that the other kingdoms will contribute to a data model. Additionally, these are the kingdoms of interest to me. So my plan for potential future hot-encoding is to only hot-encode those three kingdoms. 

__Other Naming Categoricals__
There are too many options for these to dummy at this time and I would like to sort and group on the categoricals for initial EDA. Given that these are strings, I am imputing the string "not stated" into any nulls here.

__Positional Accuracy Columns__
I choose to impute the mean into the null values for both of these columns. There are few null values and imputing the mean will at least preserve the mean. Additionally, a danger of imputing a value near the medians will put those rows inside of positional accuracies that are considered good for research. Whereas, sticking with the mean will signal "poor" positional accuracy which is the better assumption to make. 

In [72]:
data_sp["taxon_kingdom_name"].value_counts() 

Plantae      555254
Animalia     447947
Fungi         44122
Protozoa       1375
Chromista       402
Bacteria        204
Viruses          35
Archaea           1
Name: taxon_kingdom_name, dtype: int64

In [73]:
#filling categoricals with missing info with "not stated" and confirming
cat_with_null = ['species_guess', 'scientific_name','common_name', 
                 'taxon_genus_name','taxon_species_name', "taxon_kingdom_name"]
data_sp[cat_with_null] = data_sp[cat_with_null].fillna("not stated")
print(f'Number of null entries in stated columns = {data_sp[cat_with_null].isnull().sum().sum()}')

Number of null entries in stated columns = 0


In [74]:
round(data_sp[['positional_accuracy','public_positional_accuracy']].describe(),2)

,positional_accuracy,public_positional_accuracy
count,899246.00,907454.00
mean,511.41,2795.49
std,22888.17,24003.86
min,0.00,0.00
25%,5.00,5.00
50%,11.00,15.00
75%,46.00,73.00
max,9650865.00,9650865.00


In [75]:
#imputing the mean into null categories for both accuracy
PA_mean = data_sp["positional_accuracy"].mean()
PPA_mean = data_sp["public_positional_accuracy"].mean()
data_sp["positional_accuracy"] = data_sp["positional_accuracy"].fillna(PA_mean)
data_sp["public_positional_accuracy"] = data_sp["public_positional_accuracy"].fillna(PPA_mean)

In [76]:
round(data_sp[['positional_accuracy','public_positional_accuracy']].describe(),2)

,positional_accuracy,public_positional_accuracy
count,1050151.00,1050151.00
mean,511.41,2795.49
std,21179.93,22313.49
min,0.00,0.00
25%,5.00,6.00
50%,15.00,22.00
75%,173.00,550.00
max,9650865.00,9650865.00


### Final Cleaning Touches/Checks

In [77]:
# number of spicata observations in this set with necessary positional accuracy
# Goal is for this number to be at least 100
((data_sp["taxon_species_name"] == "Lobelia spicata")&(data_sp["public_positional_accuracy"]<=30)).sum()

507

I am choosing to leave nulls for urls because being able to retrieve them later from this database will be useful. I can filter those columns out in future notebooks when performing analysis. 

In [78]:
data_sp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1050151 entries, 0 to 1071655
Data columns (total 27 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   id                                1050151 non-null  int64  
 1   observed_on_string                1050151 non-null  object 
 2   observed_on                       1050151 non-null  object 
 3   time_observed_at                  1050151 non-null  object 
 4   time_zone                         1050151 non-null  object 
 5   user_id                           1050151 non-null  int64  
 6   created_at                        1050151 non-null  object 
 7   quality_grade                     1050151 non-null  object 
 8   url                               1050151 non-null  object 
 9   image_url                         1045138 non-null  object 
 10  sound_url                         3684 non-null     object 
 11  num_identification_agreements     105

In [79]:
# changing to datetimes
data_sp["time_observed_at"] = pd.to_datetime(data_sp["time_observed_at"])
data_sp["created_at"] = pd.to_datetime(data_sp["created_at"])

# Adding Important Data Columns for Analysis

### Clustering User Data According to Proximity in Time

In [80]:
# sorting data by user then by time_observed_at so "minute_diff" and "km_diff" can operate on a single user in the order
# the observations were made
# NOTE: there may be a reason to choose "created_at" or "id" instead if "time_observed_at" turns out to be poor for modeling
data_sp = data_sp.sort_values(by = ["user_id", "time_observed_at"]).reset_index(drop = True)

In [81]:
# column that calculates time difference between given row and previous row in total seconds
data_sp["minute_diff"] = data_sp["time_observed_at"].diff()
data_sp["minute_diff"] = data_sp["minute_diff"].dt.total_seconds() / 60

For each user's first entry "minute_diff", the calculated value compared to previous user's last entry doesn't make good sense. There are some options for what could be imputed there. The mean or median of that user's time differences, for example. But also perhaps 0? I'm starting with -0.001 because it's close to zero and so will act a lot like zero in this dataset if included in summary stats, but can be more easily filtered out of the dataset all together, since all actual "time differences" are zero or positive given that I constructed the set sequentially in time for each user.

In [82]:
users = data_sp["user_id"].unique() # identifies users in set --REQUIRED FOR LATER CODE
imputed_initial_time = -0.001  # change here for different imputation
for user in users:
    min_index = data_sp.index[data_sp["user_id"] == user].min()
    data_sp.loc[min_index, "minute_diff"] = imputed_initial_time

In [83]:
# summary stats with -0.001 as initial minute_diff included
round(data_sp["minute_diff"].describe(),3)

count    1.050151e+06
mean     1.183444e+03
std      5.387129e+04
min     -1.000000e-03
25%      1.000000e+00
50%      3.000000e+00
75%      1.633300e+01
max      3.626322e+07
Name: minute_diff, dtype: float64

In [84]:
# summary stats with -0.001 as initial minute_diff excluded
round(data_sp[data_sp["minute_diff"]!= -0.001]["minute_diff"].describe(),3)

count    1.049851e+06
mean     1.183782e+03
std      5.387899e+04
min      0.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      1.635000e+01
max      3.626322e+07
Name: minute_diff, dtype: float64

### Clustering User Data According to Proximity in Space (with time as an ordering category)

Using scikit learn's [haversine function ](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html) to approximate distance between two lat/lon points.

In [85]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians
def dist_in_km(df, i, s = 1): 
    '''
    Takes the index of one row in a given dataset
    and the number of rows ahead of that row in the dataset
    and computes the estimated distance in km 
    between the two latitude/longitude coordinates
    
    Arguments:
    df = dataframe
    i = index of row with first location of concern
    s = step or number of rows ahead of initial row for second location
        (default is 1--to calc distance between location in given row and next one)
    
    Returns: 
    Estimated distance in km between the two lat/lon locations 
    for the row given and the row s steps ahead of row i
    
    Example: 
    dist_in_km(data_sp, 4, 2)
    Will find the distance in km between the location specified in data_sp[4] and data_sp[4 + 2]
    '''
    location1 = [df["latitude"][i], df["longitude"][i]]
    location2 = [df["latitude"][i + s], df["longitude"][i + s]]
    loc1_rad = [radians(deg) for deg in location1]
    loc2_rad = [radians(deg) for deg in location2]
    result = haversine_distances([loc1_rad, loc2_rad])
    dist = result[0][1] * 6371  # multiply by Earth radius in km to get km
    return dist

In [86]:
# creates list of indices where new users begin
first_index_for_user = []
for user in users: # users as defined in previous section
    min_index = data_sp.index[data_sp["user_id"] == user].min()
    first_index_for_user.append(min_index)

In [87]:
# CODE RELIES ON EARLIER SORTING AND INDEXING BY USER AND TIME (or ID if chosen)
# creates distance difference column by user
# initial row for a user is imputed 
# every row afterward for that user, distance is calculated between given row and the row before
imputed_initial_dist = -0.001 # change here for different imputation
for i in range(1, data_sp.shape[0]):
    data_sp.loc[0, "km_diff"] = imputed_initial_dist #sets initial row as imputed value
    if i in first_index_for_user:
        data_sp.loc[i, "km_diff"] = imputed_initial_dist # sets all other first indices as imputed value
    else: 
        data_sp.loc[i, "km_diff"] = dist_in_km(data_sp, i-1) 
        # sets all non first indices as difference using dist_in_km function

In [88]:
# summary stats with imputed initial distance differences included
round(data_sp["km_diff"].describe(),3)

count    1050151.000
mean          19.090
std          247.831
min           -0.001
25%            0.003
50%            0.043
75%            0.598
max        19991.925
Name: km_diff, dtype: float64

In [89]:
# summary stats with imputed initial distance differences excluded
round(data_sp[data_sp["km_diff"]!= imputed_initial_dist]["km_diff"].describe(),3)

count    1049851.000
mean          19.096
std          247.867
min            0.000
25%            0.003
50%            0.043
75%            0.599
max        19991.925
Name: km_diff, dtype: float64

In [90]:
data_sp.head()

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,created_at,quality_grade,url,image_url,...,species_guess,scientific_name,common_name,taxon_kingdom_name,taxon_genus_name,taxon_species_name,geoprivacy_obscured,taxon_geoprivacy_obscured,minute_diff,km_diff
0,156253,1990-09-01 20:20:16,1990-09-01,1990-09-02 00:20:16+00:00,Eastern Time (US & Canada),317,2012-12-03 02:52:30+00:00,research,http://www.inaturalist.org/observations/156253,https://inaturalist-open-data.s3.amazonaws.com...,...,Cururu Toad,Rhinella diptycha,Cururu Toad,Animalia,Rhinella,Rhinella diptycha,0,0,-1.000000e-03,-0.001000
1,99704839,Fri May 20 1994 09:37:00 GMT-0400 (EDT),1994-05-20,1994-05-20 13:37:00+00:00,Eastern Time (US & Canada),317,2021-10-29 13:45:37+00:00,research,https://www.inaturalist.org/observations/99704839,https://inaturalist-open-data.s3.amazonaws.com...,...,Brown-headed Cowbird,Molothrus ater,Brown-headed Cowbird,Animalia,Molothrus,Molothrus ater,0,0,1.953437e+06,7492.228775
2,99705133,Fri May 20 1994 09:37:00 GMT-0400 (EDT),1994-05-20,1994-05-20 13:37:00+00:00,Eastern Time (US & Canada),317,2021-10-29 13:49:59+00:00,research,https://www.inaturalist.org/observations/99705133,https://inaturalist-open-data.s3.amazonaws.com...,...,Northern Cardinal,Cardinalis cardinalis,Northern Cardinal,Animalia,Cardinalis,Cardinalis cardinalis,0,0,0.000000e+00,0.000000
3,160735,Thu May 20 1994 15:27:35 GMT-0400 (EDT),1994-05-20,1994-05-20 19:27:35+00:00,Eastern Time (US & Canada),317,2012-12-11 20:23:37+00:00,research,http://www.inaturalist.org/observations/160735,https://inaturalist-open-data.s3.amazonaws.com...,...,Brown-headed Cowbird,Molothrus ater,Brown-headed Cowbird,Animalia,Molothrus,Molothrus ater,0,0,3.505833e+02,0.472500
4,36335283,Wed Jul 28 1999 08:24:34 GMT-0400 (EDT),1999-07-28,1999-07-28 12:24:34+00:00,Eastern Time (US & Canada),317,2019-12-04 01:50:03+00:00,research,https://www.inaturalist.org/observations/36335283,https://inaturalist-open-data.s3.amazonaws.com...,...,American Red Squirrel,Tamiasciurus hudsonicus,American Red Squirrel,Animalia,Tamiasciurus,Tamiasciurus hudsonicus,0,0,2.728377e+06,56.747939


In [91]:
#sending cleaned dataset to csv to use for EDA to cut down on runtime
#Uncomment code below to create dataset

#data_sp.to_csv("../../data/spicata_clean_500.csv")